# Initialisations

In [1]:
import pandas as pd ; pd.set_option('display.max_columns', 500) # dataframes
import numpy as np # mathsy bits
import ipywidgets as widgets # widgets

from sklearn.model_selection import train_test_split #split data into train and test sets

# feature selection + gridsearch
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import GridSearchCV

# pca
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# models
from sklearn.linear_model import Lars, ElasticNet, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

import joblib # saving models
from datetime import datetime # get time for labelling saved models

# visualization
import matplotlib.pyplot as plt
%matplotlib inline

import mpld3 # hover-over labels for plots
mpld3.disable_notebook()

train_scores = dict() ; test_scores  = dict() #to hold scores for viewing at the end

In [2]:
def grid_search_return_hp_dict(model, X_train_gs, y_train_gs, param_grid):
    """Run GridSearchCV on given model with defined param_grid, return best params as dict.
    Pre-assign sklearn model to variable. Pass param_grid as dict."""
    print(str(model).split(sep='(')[0], "grid search.")
    GS = GridSearchCV(model, param_grid, n_jobs=2)
    GS.fit(X_train_gs, y_train_gs)
    print(GS.best_params_)
    return(GS.best_params_)

def produce_exp_vs_pred_df(features_list, model, codename):
    """Pass features_list as a string"""
    pred_list = []
    for i in range(0, len(pca_features)):
        prediction = model.predict(np.array(pca_features.iloc[i]).reshape(1, -1))
        pred_list.append(prediction)
    train_scores.update({codename : model.score(X_train, y_train)})
    test_scores.update({codename : model.score(X_test, y_test)})
    
    print('Training Score:\t', model.score(X_train, y_train))
    print('Testing Score:\t', model.score(X_test, y_test))
    
    exp_vs_calc = pd.DataFrame(constants_first)
    exp_vs_calc['Predicted'] = pred_list
    exp_vs_calc.rename({'Kh_first':'Experimental'}, inplace=True, axis=1)
    
    now = datetime.now()

    dt_string = now.strftime("_%d_%m_%Y_%H_%M_%S")
    filename = "models/PCA_DRAGON_VPAS/" + str(model).split(sep='(')[0] + "_pca" + dt_string + ".joblib"
    print(str(model).split(sep='(')[0], "run at:", now, ". Saving to", filename)
    
    joblib.dump(model, filename)
    return exp_vs_calc

def prediction_plot_scores(model_func, pred_df):
    """Print train+test scores, then plot scatter of predicted vs actual HLCs. Uses {X/y}_{train/test},
    redefining these variables will change the output."""
    #print('Training Score:\t', model_func.score(X_train, y_train))
    #print('Testing Score:\t', model_func.score(X_test, y_test))
    fig = plt.figure(figsize=(10, 10))
    scatter = plt.scatter(pred_df['Experimental'], pred_df['Predicted'])
    plt.xlabel('Experimental')
    plt.ylabel('Predicted')
    plt.title('%s predictions of HLCs' %str(model_func).split(sep='(')[0])
    labels = ['{}'.format(i) for i in species_names]
    tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
    mpld3.plugins.connect(fig, tooltip)
    plt.plot([-30, 10], [-30, 10], c='red')
    plt.show()

# DRAGON descriptors
## Bringing in data and splitting it into parts

In [3]:
csv = pd.read_csv('filtered_organics_desc_kh.csv') # contains VP/AS HLCs

print("Input Shape", csv.shape) #input shape

csv.dropna(axis=0, inplace=True)

print("Removed NaN, new shape", csv.shape) #removed NaN shape

smiles_strings = csv.pop('Unnamed: 0')
species_names = csv.pop('0')
constants_mean = csv.pop('Kh_mean')
constants_first = csv.pop('Kh_first')

varying_columns = csv[['Varying_1', 'Varying_2', 'Varying_3', 'Varying_4',
 'Varying_5', 'Varying_6', 'Varying_7', 'Varying_8', 'Varying_9']] # popping one-hot encoding columns

dragon_features = csv.drop(['Varying_1', 'Varying_2', 'Varying_3', 'Varying_4',
 'Varying_5', 'Varying_6', 'Varying_7', 'Varying_8', 'Varying_9'], axis=1) #seperating features



Input Shape (2075, 1480)
Removed NaN, new shape (2068, 1480)


## PCA

In [4]:
pca_standard_x = StandardScaler().fit_transform(dragon_features)
pca = PCA(n_components=15)
pca_components = pca.fit_transform(pca_standard_x)

pca_features = pd.DataFrame(data = pca_components)
pca_df = pd.concat([pca_features, constants_first])

In [5]:
pca_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-36.860026,6.900018,0.607076,5.243564,5.849608,14.852287,-3.153825,-6.243518,-1.884733,-1.025200,-2.661705,0.114541,0.587299,-1.237348,-0.811040
1,-32.152179,6.049895,-1.742206,5.710479,4.939049,13.700101,-2.175988,-5.002545,-2.177486,0.829409,-2.981593,0.166138,1.022370,-0.539167,-3.276557
2,-26.016767,5.078370,-3.387507,3.884430,2.708417,8.443312,0.474228,-2.812891,-1.755071,-0.338023,-1.292612,-0.222576,2.286590,1.454164,-0.511096
3,-20.942300,4.029370,-5.477529,2.465034,0.994654,4.380148,2.054669,-1.229627,-1.665471,0.351866,-0.785901,-1.342538,1.711902,2.691001,-1.167079
4,-21.649762,4.491667,-5.045159,2.773506,1.459018,5.258322,2.794104,-1.845114,-1.970487,-1.530491,0.825757,0.075165,4.307395,2.411842,1.646211


In [6]:
X_train, X_test, y_train, y_test = train_test_split(pca_features, constants_first, test_size=0.3)

## GradientBoosting
### Model-chosen features

In [7]:
gbr_param_grid = {
    'n_estimators': [500, 1000, 2000],
    'max_depth': [2, 4, 6],
    'min_samples_leaf': [3, 5, 9, 17],
    'learning_rate': [0.1, 0.05, 0.02],
    'max_features': [1.0, 0.3, 0.1],
    'loss': ['ls', 'lad', 'huber']
} #hyperparameters for each gbr grid search

In [8]:
best_hp = grid_search_return_hp_dict(GradientBoostingRegressor(),
                                     X_train, y_train, gbr_param_grid)

gbr_model = GradientBoostingRegressor(**best_hp)
gbr_model.fit(X_train, y_train)

exp_vs_calc_gbr_model = produce_exp_vs_pred_df('top_15_features_model', gbr_model,
                                         "GradBoost, model-chosen features")

GradientBoostingRegressor grid search.


C:\Users\jtd1g16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'learning_rate': 0.02, 'loss': 'huber', 'max_depth': 6, 'max_features': 0.3, 'min_samples_leaf': 9, 'n_estimators': 2000}
Training Score:	 0.9857430273654677
Testing Score:	 0.8176338827919104
GradientBoostingRegressor run at: 2020-01-17 14:16:42.240120 . Saving to models/PCA_DRAGON_VPAS/GradientBoostingRegressor_pca_17_01_2020_14_16_42.joblib


### F-regression features

In [10]:
best_hp = grid_search_return_hp_dict(GradientBoostingRegressor(),
                                     X_train, y_train, gbr_param_grid)

gbr_freg = GradientBoostingRegressor(**best_hp)
gbr_freg.fit(X_train, y_train)

exp_vs_calc_gbr_freg = produce_exp_vs_pred_df('top_15_features_kbest', gbr_freg,
                                                    "GradBoost, f_reg features")

GradientBoostingRegressor grid search.


C:\Users\jtd1g16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'learning_rate': 0.02, 'loss': 'huber', 'max_depth': 6, 'max_features': 0.3, 'min_samples_leaf': 3, 'n_estimators': 2000}
Training Score:	 0.9964791587515248
Testing Score:	 0.8090169824873324
GradientBoostingRegressor run at: 2020-01-17 15:36:43.683301 . Saving to models/PCA_DRAGON_VPAS/GradientBoostingRegressor_pca_17_01_2020_15_36_43.joblib


### Mutual info features

In [ ]:
best_hp = grid_search_return_hp_dict(GradientBoostingRegressor(),
                                     X_train, y_train, gbr_param_grid)

gbr_mu = GradientBoostingRegressor(**best_hp)
gbr_mu.fit(X_train, y_train)

exp_vs_calc_gbr_mu = produce_exp_vs_pred_df('top_15_features_mutual_info', gbr_mu,
                                            "GradBoost, Mutual Info")

### LASSO features

In [ ]:
best_hp = grid_search_return_hp_dict(GradientBoostingRegressor(), 
                                     X_train, y_train, gbr_param_grid)

gbr_lasso = GradientBoostingRegressor(**best_hp)
gbr_lasso.fit(X_train, y_train)
exp_vs_calc_gbr_lasso = produce_exp_vs_pred_df('top_15_features_lasso', gbr_lasso,
                                               "LASSO-features, GBR model")

## Decision Tree
### Model-Chosen Features

In [ ]:
dtr_param_grid = {
    'criterion': ['mse', 'friedman_mse', 'mae'], #function measuring quality of a split
    'max_depth': [5, 10, 20] , # max depth of tree
    'max_features': [5, 10, 15], # N features to be considered when looking for split
    'max_leaf_nodes': [None, 10, 15, 20, 30], #grows tree with N nodes in best-first fashion
    'min_impurity_decrease': [0.0, 0.1],
    'min_samples_leaf': [1, 2, 3], 
    'min_samples_split': [1.0, 2, 3],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.5] , 
    'splitter': ['best', 'random']
} #hyperparameters for dtr gridsearch

In [ ]:
best_hp = grid_search_return_hp_dict(DecisionTreeRegressor(), X_train, y_train, dtr_param_grid)

dtr_model = DecisionTreeRegressor(**best_hp)

dtr_model.fit(X_train, y_train)

exp_vs_calc_dtr_model = produce_exp_vs_pred_df('top_15_features_model', dtr_model,
                                         "Decision Tree, model-chosen features")

### F-regression features

In [ ]:
best_hp = grid_search_return_hp_dict(DecisionTreeRegressor(), X_train, y_train, dtr_param_grid)

dtr_freg = DecisionTreeRegressor(**best_hp)

dtr_freg.fit(X_train, y_train)

exp_vs_calc_dtr_freg = produce_exp_vs_pred_df('top_15_features_kbest', dtr_freg,
                                         "Decision Tree, freg features")

### Mutual info features

In [ ]:
best_hp = grid_search_return_hp_dict(DecisionTreeRegressor(), X_train, y_train, dtr_param_grid)

dtr_mu = DecisionTreeRegressor(**best_hp)

dtr_mu.fit(X_train, y_train)

exp_vs_calc_dtr_mu = produce_exp_vs_pred_df('top_15_features_mutual_info', dtr_mu,
                                         "Decision Tree, mutual info features")

### LASSO features

In [ ]:
best_hp = grid_search_return_hp_dict(DecisionTreeRegressor(), X_train, y_train, dtr_param_grid)

dtr_lasso = DecisionTreeRegressor(**best_hp)

dtr_lasso.fit(X_train, y_train)

exp_vs_calc_dtr_lasso = produce_exp_vs_pred_df('top_15_features_lasso', dtr_lasso,
                                         "Decision Tree, lasso features")

## AdaBoost Regressor
### Model Chosen Features

In [ ]:
base_models = [ExtraTreesRegressor(n_estimators= 5,
                                   criterion= 'mse',
                                   max_features = 'log2'),
               RandomForestRegressor(n_estimators= 5,
                                     criterion= 'mse',
                                     max_features = 'sqrt',
                                     min_samples_split = 3),
              GradientBoostingRegressor(),
              DecisionTreeRegressor(),
              Lars(),
              ElasticNet()]

ada_param_grid = {
    'base_estimator' : base_models, 
    'learning_rate' : [0.3, 0.5, 0.8, 1], 
    'loss' : ['linear', 'square', 'exponential'], 
    'n_estimators' : [50, 100]
} #hyperparameters for adaboost gridsearch

In [ ]:
X_train, X_test = X_train_main[top_15_features_model], X_test_main[top_15_features_model]

best_hp = grid_search_return_hp_dict(AdaBoostRegressor(), X_train, y_train, ada_param_grid)

ada_model = AdaBoostRegressor(**best_hp)

ada_model.fit(X_train, y_train)

exp_vs_calc_ada_model = produce_exp_vs_pred_df('top_15_features_model', ada_model,
                                               "AdaBoost, model features")

### F-regression features

In [ ]:
X_train, X_test = X_train_main[top_15_features_kbest], X_test_main[top_15_features_kbest]

best_hp = grid_search_return_hp_dict(AdaBoostRegressor(), X_train, y_train, ada_param_grid)

ada_freg = AdaBoostRegressor(**best_hp)

ada_freg.fit(X_train, y_train)

exp_vs_calc_ada_freg = produce_exp_vs_pred_df('top_15_features_kbest', ada_freg,
                                               "AdaBoost, freg features")

### Mutual info features

In [ ]:
X_train, X_test = X_train_main[top_15_features_mutual_info], X_test_main[top_15_features_mutual_info]

best_hp = grid_search_return_hp_dict(AdaBoostRegressor(), X_train, y_train, ada_param_grid)

ada_mu = AdaBoostRegressor(**best_hp)

ada_mu.fit(X_train, y_train)

exp_vs_calc_ada_mu = produce_exp_vs_pred_df('top_15_features_mutual_info', ada_mu,
                                               "AdaBoost, mutual info features")

### LASSO features

In [ ]:
X_train, X_test = X_train_main[top_15_features_lasso], X_test_main[top_15_features_lasso]

best_hp = grid_search_return_hp_dict(AdaBoostRegressor(), X_train, y_train, ada_param_grid)

ada_lasso = AdaBoostRegressor(**best_hp)

ada_lasso.fit(X_train, y_train)

exp_vs_calc_ada_lasso = produce_exp_vs_pred_df('top_15_features_lasso', ada_lasso,
                                               "AdaBoost, model features")

## LASSO
### Model Chosen Features

In [ ]:
lasso_param_grid = {
    'alpha':[0.2, 0.4, 0.6, 0.8],
    'max_iter':[1000, 5000, 10000, 50000],
    'selection':['cyclic', 'random']
}

In [ ]:
X_train, X_test = X_train_main[top_15_features_model], X_test_main[top_15_features_model]
best_hp = grid_search_return_hp_dict(Lasso(), X_train, y_train, lasso_param_grid)
lasso_model = Lasso(**best_hp)
lasso_model.fit(X_train, y_train)

exp_vs_calc_lasso_model = produce_exp_vs_pred_df('top_15_features_model', lasso_model,
                                                 "LASSO, model features")

### F-regression features

In [ ]:
X_train, X_test = X_train_main[top_15_features_kbest], X_test_main[top_15_features_kbest]
best_hp = grid_search_return_hp_dict(Lasso(), X_train, y_train, lasso_param_grid)
lasso_freg = Lasso(**best_hp)
lasso_freg.fit(X_train, y_train)

exp_vs_calc_lasso_freg = produce_exp_vs_pred_df('top_15_features_kbest', lasso_freg,
                                                 "LASSO, f-reg features")

### Mutual info features

In [ ]:
X_train, X_test = X_train_main[top_15_features_mutual_info], X_test_main[top_15_features_mutual_info]
best_hp = grid_search_return_hp_dict(Lasso(), X_train, y_train, lasso_param_grid)
lasso_mu = Lasso(**best_hp)
lasso_mu.fit(X_train, y_train)

exp_vs_calc_lasso_mu = produce_exp_vs_pred_df('top_15_features_mutual_info', lasso_mu,
                                                 "LASSO, mutual info features")

### LASSO Features

In [ ]:
X_train, X_test = X_train_main[top_15_features_lasso], X_test_main[top_15_features_lasso]
best_hp = grid_search_return_hp_dict(Lasso(), X_train, y_train, lasso_param_grid)
lasso_lasso = Lasso(**best_hp)
lasso_lasso.fit(X_train, y_train)

exp_vs_calc_lasso_lasso = produce_exp_vs_pred_df('top_15_features_lasso', lasso_lasso,
                                                 "LASSO, lasso features")

## Random Forest Regressor
### Model Chosen Features

In [ ]:
random_forest_param_grid = {
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600]
}

In [ ]:
X_train, X_test = X_train_main[top_15_features_model], X_test_main[top_15_features_model]
best_hp = grid_search_return_hp_dict(RandomForestRegressor(),
                                     X_train, y_train, random_forest_param_grid)
random_forest_model = RandomForestRegressor(**best_hp)
random_forest_model.fit(X_train, y_train)
exp_vs_calc_random_forest_model = produce_exp_vs_pred_df('top_15_features_model', random_forest_model,
                                                         "RandomForest, model features")

### F-regression features

In [ ]:
X_train, X_test = X_train_main[top_15_features_kbest], X_test_main[top_15_features_kbest]
best_hp = grid_search_return_hp_dict(RandomForestRegressor(),
                                     X_train, y_train, random_forest_param_grid)
random_forest_freg = RandomForestRegressor(**best_hp)
random_forest_freg.fit(X_train, y_train)
exp_vs_calc_random_forest_freg = produce_exp_vs_pred_df('top_15_features_kbest', random_forest_freg,
                                                         "RandomForest, freg features")

### Mutual info features

In [ ]:
X_train, X_test = X_train_main[top_15_features_mutual_info], X_test_main[top_15_features_mutual_info]
best_hp = grid_search_return_hp_dict(RandomForestRegressor(),
                                     X_train, y_train, random_forest_param_grid)
random_forest_mu = RandomForestRegressor(**best_hp)
random_forest_mu.fit(X_train, y_train)
exp_vs_calc_random_forest_mu = produce_exp_vs_pred_df('top_15_features_mutual_info', random_forest_mu,
                                                         "RandomForest, mutual info features")

### LASSO features

In [ ]:
X_train, X_test = X_train_main[top_15_features_lasso], X_test_main[top_15_features_lasso]
best_hp = grid_search_return_hp_dict(RandomForestRegressor(),
                                     X_train, y_train, random_forest_param_grid)
random_forest_lasso = RandomForestRegressor(**best_hp)
random_forest_lasso.fit(X_train, y_train)
exp_vs_calc_random_forest_lasso = produce_exp_vs_pred_df('top_15_features_lasso', random_forest_lasso,
                                                         "RandomForest, lasso features")

## LARS - Least angle regression
### Model chosen features

In [ ]:
X_train, X_test = X_train_main[top_15_features_model], X_test_main[top_15_features_model]

lars_model = Lars(fit_intercept=False, n_nonzero_coefs=np.inf)
lars_model.fit(X_train, y_train)
exp_vs_calc_lars_model = produce_exp_vs_pred_df('top_15_features_model', lars_model, "LARS model features")

### F-regression features

In [ ]:
X_train, X_test = X_train_main[top_15_features_kbest], X_test_main[top_15_features_kbest]

lars_freg = Lars(fit_intercept=False, n_nonzero_coefs=np.inf)
lars_freg.fit(X_train, y_train)
exp_vs_calc_lars_freg = produce_exp_vs_pred_df('top_15_features_kbest', lars_freg, "LARS freg features")

### Mutual info features

In [ ]:
X_train, X_test = X_train_main[top_15_features_mutual_info], X_test_main[top_15_features_mutual_info]

lars_mu = Lars(fit_intercept=False, n_nonzero_coefs=np.inf)
lars_mu.fit(X_train, y_train)
exp_vs_calc_lars_mu = produce_exp_vs_pred_df('top_15_features_mutual_info', lars_mu, "LARS mutual info features")

### LASSO features

In [ ]:
X_train, X_test = X_train_main[top_15_features_lasso], X_test_main[top_15_features_lasso]

lars_lasso = Lars(fit_intercept=False, n_nonzero_coefs=np.inf)
lars_lasso.fit(X_train, y_train)
exp_vs_calc_lars_lasso = produce_exp_vs_pred_df('top_15_features_lasso', lars_lasso, "LARS lasso features")

## Elastic Net (linear regression l1 l2 norm regularization)
### Model chosen features

In [ ]:
net_param_grid = {
    'alpha':[0.1, 0.3, 0.5, 0.8, 1.0],
    'l1_ratio':[0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    'fit_intercept':[True, False],
    'max_iter': [1000, 3000, 5000],
    'tol':[0.0001, 0.001, 0.01],
    'selection':['cyclic', 'random']
}

In [ ]:
X_train, X_test = X_train_main[top_15_features_model], X_test_main[top_15_features_model]
best_hp = grid_search_return_hp_dict(ElasticNet(), X_train, y_train, net_param_grid)

net_model = ElasticNet(**best_hp)
net_model.fit(X_train, y_train)
exp_vs_calc_net_model = produce_exp_vs_pred_df('top_15_features_model', net_model, "ElasticNet model features")

### F regression features

In [ ]:
X_train, X_test = X_train_main[top_15_features_kbest], X_test_main[top_15_features_kbest]
best_hp = grid_search_return_hp_dict(ElasticNet(), X_train, y_train, net_param_grid)

net_freg = ElasticNet(**best_hp)
net_freg.fit(X_train, y_train)
exp_vs_calc_net_freg = produce_exp_vs_pred_df('top_15_features_kbest', net_freg, "ElasticNet freg features")

### Mutual info features

In [ ]:
X_train, X_test = X_train_main[top_15_features_mutual_info], X_test_main[top_15_features_mutual_info]
best_hp = grid_search_return_hp_dict(ElasticNet(), X_train, y_train, net_param_grid)

net_mu = ElasticNet(**best_hp)
net_mu.fit(X_train, y_train)
exp_vs_calc_net_mu = produce_exp_vs_pred_df('top_15_features_mutual_info', net_mu, "ElasticNet mutual info features")

### LASSO features

In [ ]:
X_train, X_test = X_train_main[top_15_features_lasso], X_test_main[top_15_features_lasso]
best_hp = grid_search_return_hp_dict(ElasticNet(), X_train, y_train, net_param_grid)

net_lasso = ElasticNet(**best_hp)
net_lasso.fit(X_train, y_train)
exp_vs_calc_net_lasso = produce_exp_vs_pred_df('top_15_features_lasso', net_lasso, "ElasticNet lasso features")

# Scores and visualisation

In [ ]:
exp_vs_calc_df_list = []
model_list = []
for var in dir():
    if isinstance(eval(var), pd.core.frame.DataFrame):
        if var[0:3] == 'exp':
            exp_vs_calc_df_list.append(var)
            model_list.append(var[12:])
exp_vs_calc_model_dict = dict(zip(model_list, exp_vs_calc_df_list))

In [ ]:
joblib.dump(train_scores, "train_scores.joblib")
joblib.dump(test_scores, "test_scores.joblib")

In [ ]:
display("Training Scores", train_scores, "Testing Scores", test_scores)
mpld3.enable_notebook() #must enable
@widgets.interact()

def show_pred_vs_actual(Model = model_list):
    pred_df = eval(exp_vs_calc_model_dict[Model])
    model = eval(Model)
    prediction_plot_scores(model, pred_df)